In [1]:
# Dependencies
from bs4 import BeautifulSoup
from splinter import Browser
from flask import Flask, render_template, redirect

import requests
import pymongo
import pandas as pd

In [2]:
executable_path = {'executable_path': '../chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

### NASA Mars News

In [3]:
url = 'https://mars.nasa.gov/news/'
browser.visit(url)
html=browser.html
soup=BeautifulSoup(html,'html.parser')

In [4]:
# Retrieve the latest news title
news_title=soup.find_all('div', class_='content_title')[1].text
print(news_title)

NASA's Perseverance Rover Gives High-Definition Panoramic View of Landing Site


In [5]:
new_p=soup.find_all('div', class_='article_teaser_body')[0].text
print(new_p)

A 360-degree panorama taken by the rover’s Mastcam-Z instrument will be discussed during a public video chat this Thursday.


### JPL Mars Space Images - Featured Image

In [6]:
jpl_url="https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html"
browser.visit(jpl_url)
html = browser.html
soup = BeautifulSoup(html, "html.parser")

In [7]:
img_url=soup.find_all('img', class_='headerimage')[0]['src']
featured_image_url=jpl_url+img_url
featured_image_url

'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.htmlimage/featured/mars3.jpg'

### Mars Facts

In [8]:
facts_url="https://space-facts.com/mars/"
browser.visit(facts_url)
html = browser.html
soup = BeautifulSoup(html, "html.parser")

In [9]:
tb=pd.read_html('https://space-facts.com/mars/')
facts_tb=tb[0]
facts_tb.rename(columns={0:'Description',1:'Mars'},inplace=True)
facts_tb.set_index('Description',inplace=True)
facts_tb

,Mars
Description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [10]:
facts_tb_html=facts_tb.to_html()
facts_tb_html

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n    </tr>\n    <tr>\n      <th>Description</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period:</th>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>Surface Temperature:</th>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>First Record:</th>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>Recorded By:</th>\n      <td>Egyptian astronomers</td>\n    </tr>\n

### Mars Hemispheres

In [44]:
base_url="https://astrogeology.usgs.gov"
search_url="/search/results?q=hemisphere+enhanced&k1=target&v1=Mars/"
browser.visit(base_url+search_url)
html = browser.html
soup = BeautifulSoup(html, "html.parser")

In [45]:
title_list=[]
search_list=[]

In [46]:
hemi_url=soup.find_all('div', class_='description')
print(hemi_url[0].find('a')['href'])
print(hemi_url[0].find('h3').text)

/search/map/Mars/Viking/cerberus_enhanced
Cerberus Hemisphere Enhanced


In [63]:
title_list=[hemi.find('h3').text for hemi in hemi_url]
title_list
search_list=[hemi.find('a')['href'] for hemi in hemi_url]
print(search_list[0])
print(f"{base_url}{search_list[0]}")

/search/map/Mars/Viking/cerberus_enhanced
https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced
https://astrogeology.usgs.gov/


In [83]:
hemisphere_image_urls=[]
for s_url in search_list:
    tmp_url=base_url+s_url
    print(tmp_url)
    browser.visit(tmp_url)
    html = browser.html
    soup = BeautifulSoup(html, "html.parser")
    hemi_img_dct={}
    title=soup.find('h2',class_='title').text
    #print(title)
    img_url=soup.find_all('li')[1].a['href']
    #print(img_url)
    hemi_img_dct["title"]=title
    hemi_img_dct["img_url"]=img_url
    hemisphere_image_urls.append(hemi_img_dct)

print(hemisphere_image_urls)

https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced
https://astrogeology.usgs.gov/search/map/Mars/Viking/schiaparelli_enhanced
https://astrogeology.usgs.gov/search/map/Mars/Viking/syrtis_major_enhanced
https://astrogeology.usgs.gov/search/map/Mars/Viking/valles_marineris_enhanced
[{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif'}]
